This notebook is intended to compare the Gaia DR3 and VLBI astrometry.

In [1]:
import numpy as np
from numpy import concatenate, cos, pi, sin, sqrt

from astropy.table import Table, join
from astropy import units as u
from astropy.time import Time

Load the PSR$\pi$ table, which contains 61 pulsars.

In [2]:
vlbi_table = Table.read("../data/psrpi.dat", format="ascii")


vlbi_table["ra"].unit = u.deg
vlbi_table["dec"].unit = u.deg
vlbi_table["pmra"].unit = u.mas / u.yr
vlbi_table["pmdec"].unit = u.mas / u.yr
vlbi_table["plx"].unit = u.mas
vlbi_table["ra_err"].unit = u.mas
vlbi_table["dec_err"].unit = u.mas
vlbi_table["pmra_err"].unit = u.mas / u.yr
vlbi_table["pmdec_err"].unit = u.mas / u.yr
vlbi_table["plx_err"].unit = u.mas


Load the Gaia DR3 pulsar table.

In [3]:
gaia_table = Table.read("../data/edr3_psrcatv1.67.fits")

gaia_table = gaia_table[
    "name",
    "source_id",
    "ra",
    "ra_error",
    "dec",
    "dec_error",
    "parallax",
    "parallax_error",
    "pmra",
    "pmra_error",
    "pmdec",
    "pmdec_error",
]

gaia_table.rename_columns(
    [
        "name",
        "ra_error",
        "dec_error",
        "parallax",
        "parallax_error",
        "pmra_error",
        "pmdec_error",
    ],
    ["psr_name", "ra_err", "dec_err", "plx", "plx_err", "pmra_err", "pmdec_err"],
)

gaia_table["ra"].unit = u.deg
gaia_table["dec"].unit = u.deg
gaia_table["pmra"].unit = u.mas / u.yr
gaia_table["pmdec"].unit = u.mas / u.yr
gaia_table["plx"].unit = u.mas
gaia_table["ra_err"].unit = u.mas
gaia_table["dec_err"].unit = u.mas
gaia_table["pmra_err"].unit = u.mas / u.yr
gaia_table["pmdec_err"].unit = u.mas / u.yr
gaia_table["plx_err"].unit = u.mas


Find the common pulsars.

In [4]:
com_table = join(gaia_table, vlbi_table, keys="psr_name", table_names=["g", "v"])

Computed the differences.

In [5]:
gaia_epoch = Time("2016.0", format="jyear").mjd
# 57388.5

# Epoch difference in Julian year
dt_y = -(com_table["pos_epoch"] - gaia_epoch) / 365.25 * u.yr

# Cos(decl.) factor for the difference in R.A.
dec_fac = cos(np.deg2rad(com_table["dec_v"].value))

dra = (
    com_table["ra_g"] - com_table["ra_v"]
) * dec_fac - dt_y * com_table["pmra_v"]

ddec = com_table["dec_g"] - com_table["dec_v"] - \
    dt_y * com_table["pmdec_v"]

dra = dra.to(u.mas)
ddec = ddec.to(u.mas)

com_table.add_columns([dra, ddec], names=["dra", "ddec"])

com_table["ra_err_v"] = com_table["ra_err_v"] * dec_fac

dra_err = sqrt(
    com_table["ra_err_g"] ** 2
    + com_table["ra_err_v"] ** 2
    + (dt_y * com_table["pmra_err_v"]).value ** 2
)

ddec_err = sqrt(
    com_table["dec_err_g"] ** 2
    + com_table["dec_err_v"] ** 2
    + (dt_y * com_table["pmdec_err_v"]).value ** 2
)

com_table.add_columns([dra_err, ddec_err], names=["dra_err", "ddec_err"])

Format the results.

In [6]:
for i in range(len(com_table)):

    print("----------------------------------------------------")
    print("PSR", com_table[i]["psr_name"])
    print("----------------------------------------------------")
    print("{:20s}  :  {:>8s}  |  {:>8s}".format("Parameters", "VLBI", "Gaia"))

    print("Position comparison")
    print("{:20s}  :  {:>8.3f}  |  {:>8.3f}".format(
        "ra_err", com_table[i]["ra_err_v"], com_table[i]["ra_err_g"]))
    print("{:20s}  :  {:>8.3f}  |  {:>8.3f}".format(
        "dec_err", com_table[i]["dec_err_v"], com_table[i]["dec_err_g"]))
    print("{:20s}  :  {:>8.3f} +/-  {:>8.3f}".format(
        "Delta_ra", com_table[i]["dra"], com_table[i]["dra_err"]))
    print("{:20s}  :  {:>8.3f} +/-  {:>8.3f}".format(
        "Delta_dec", com_table[i]["ddec"], com_table[i]["ddec_err"]))

    print("Proper motion comparison")
    print("{:20s}  :  {:>8.3f}  |  {:>8.3f}".format(
        "pmra", com_table[i]["pmra_v"], com_table[i]["pmra_g"]))
    print("{:20s}  :  {:>8.3f}  |  {:>8.3f}".format(
        "pmra_err", com_table[i]["pmra_err_v"], com_table[i]["pmra_err_g"]))

    print("{:20s}  :  {:>8.3f}  |  {:>8.3f}".format(
        "pmdec", com_table[i]["pmdec_v"], com_table[i]["pmdec_g"]))
    print("{:20s}  :  {:>8.3f}  |  {:>8.3f}".format(
        "pmdec_err", com_table[i]["pmdec_err_v"], com_table[i]["pmdec_err_g"]))

    print("Parallax comparison")
    print("{:20s}  :  {:>8.3f}  |  {:>8.3f}".format(
        "plx", com_table[i]["plx_v"], com_table[i]["plx_g"]))
    print("{:20s}  :  {:>8.3f}  |  {:>8.3f}".format(
        "plx_err", com_table[i]["plx_err_v"], com_table[i]["plx_err_g"]))


----------------------------------------------------
PSR J0614+2229
----------------------------------------------------
Parameters            :      VLBI  |      Gaia
Position comparison
ra_err                :     1.281  |     0.544
dec_err               :     1.000  |     0.535
Delta_ra              :   361.408 +/-     1.404
Delta_dec             :   160.566 +/-     1.165
Proper motion comparison
pmra                  :    -0.230  |    -0.531
pmra_err              :     0.050  |     0.678
pmdec                 :    -1.220  |    -0.872
pmdec_err             :     0.070  |     0.461
Parallax comparison
plx                   :     0.280  |    -1.224
plx_err               :     0.030  |     0.678
----------------------------------------------------
PSR J1012+5307
----------------------------------------------------
Parameters            :      VLBI  |      Gaia
Position comparison
ra_err                :     0.540  |     0.204
dec_err               :     1.000  |     0.204
Delta_ra     